In [36]:
#Importing libraries
import pandas as pd
import plotly.express as px
import re
import numpy as np

In [37]:
#reading movie dataset
movies_df = pd.read_csv('movies.dat',
                        delimiter='::', engine= 'python', header=None,
                        names=['title', 'genres'], encoding='ISO-8859-1').rename_axis('movie_id')

movies_df.head()


,title,genres
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy


In [38]:

movies_df['genres'] = movies_df['genres'].str.split('|')
movies_df.head()


,title,genres
movie_id,,
1,Toy Story (1995),"[Animation, Children's, Comedy]"
2,Jumanji (1995),"[Adventure, Children's, Fantasy]"
3,Grumpier Old Men (1995),"[Comedy, Romance]"
4,Waiting to Exhale (1995),"[Comedy, Drama]"
5,Father of the Bride Part II (1995),[Comedy]


In [39]:
def extract_year(title):
    match = re.search(r'\((\d{4})\)', title)#re.search(r'\((\d{4})\)$', title)
    if match:
        return match.group(1)
    else:
        return None
movies_df['year'] = movies_df['title'].apply(extract_year)
movies_df.head()

,title,genres,year
movie_id,,,
1,Toy Story (1995),"[Animation, Children's, Comedy]",1995
2,Jumanji (1995),"[Adventure, Children's, Fantasy]",1995
3,Grumpier Old Men (1995),"[Comedy, Romance]",1995
4,Waiting to Exhale (1995),"[Comedy, Drama]",1995
5,Father of the Bride Part II (1995),[Comedy],1995


In [40]:
movies_df['movie_name'] = movies_df['title'].apply(lambda title: re.sub(r'\s\(\d{4}\)$', '', title))

movies_df

,title,genres,year,movie_name
movie_id,,,,
1,Toy Story (1995),"[Animation, Children's, Comedy]",1995,Toy Story
2,Jumanji (1995),"[Adventure, Children's, Fantasy]",1995,Jumanji
3,Grumpier Old Men (1995),"[Comedy, Romance]",1995,Grumpier Old Men
4,Waiting to Exhale (1995),"[Comedy, Drama]",1995,Waiting to Exhale
5,Father of the Bride Part II (1995),[Comedy],1995,Father of the Bride Part II
...,...,...,...,...
3948,Meet the Parents (2000),[Comedy],2000,Meet the Parents
3949,Requiem for a Dream (2000),[Drama],2000,Requiem for a Dream
3950,Tigerland (2000),[Drama],2000,Tigerland


In [41]:
dim_movies = movies_df[['movie_name', 'genres', 'year']]
dim_movies.head()

,movie_name,genres,year
movie_id,,,
1,Toy Story,"[Animation, Children's, Comedy]",1995
2,Jumanji,"[Adventure, Children's, Fantasy]",1995
3,Grumpier Old Men,"[Comedy, Romance]",1995
4,Waiting to Exhale,"[Comedy, Drama]",1995
5,Father of the Bride Part II,[Comedy],1995


In [42]:
dim_users = pd.read_csv(f'users.dat',
                       delimiter='::', engine='python', header=None,
                       names=['user_id', 'gender', 'age', 'occupation', 'zip_code'],encoding='ISO-8859-1')
dim_users.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [43]:
readme_text = np.array(open('README').read().splitlines())
start_index = np.flatnonzero(np.core.defchararray.find(readme_text,'Occupation is chosen')!=-1)[0]
end_index = np.flatnonzero(np.core.defchararray.find(readme_text,'MOVIES FILE DESCRIPTION')!=-1)[0]
occupation_list = [x.split('"')[1] for x in readme_text[start_index:end_index][2:-1].tolist()]
occupation_dict = dict(zip(range(len(occupation_list)), occupation_list))
dim_users['occupation'] = dim_users['occupation'].replace(occupation_dict)
dim_users.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,K-12 student,48067
1,2,M,56,self-employed,70072
2,3,M,25,scientist,55117
3,4,M,45,executive/managerial,02460
4,5,M,25,writer,55455


In [44]:
ratings_df = pd.read_csv('ratings.dat',
                        delimiter='::', engine= 'python', header=None,
                        names=['user_id', 'movie_id', 'user_rating', 'time'], encoding='ISO-8859-1')
ratings_df.head()


,user_id,movie_id,user_rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [47]:
average_ratings = ratings_df.groupby('movie_id')['user_rating'].mean().round(1)
dim_movies = pd.merge(dim_movies, average_ratings, on='movie_id', how='left').rename(columns={'user_rating': 'average_rating'})
dim_movies.head()

,movie_name,genres,year,average_rating
movie_id,,,,
1,Toy Story,"[Animation, Children's, Comedy]",1995,4.1
2,Jumanji,"[Adventure, Children's, Fantasy]",1995,3.2
3,Grumpier Old Men,"[Comedy, Romance]",1995,3.0
4,Waiting to Exhale,"[Comedy, Drama]",1995,2.7
5,Father of the Bride Part II,[Comedy],1995,3.0


In [48]:
fact_table = pd.merge(pd.merge(ratings_df, dim_movies, on='movie_id', how='left'), dim_users, on='user_id', how='left')
fact_table.head()

,user_id,movie_id,user_rating,time,movie_name,genres,year,average_rating,gender,age,occupation,zip_code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest,[Drama],1975,4.4,F,1,K-12 student,48067
1,1,661,3,978302109,James and the Giant Peach,"[Animation, Children's, Musical]",1996,3.5,F,1,K-12 student,48067
2,1,914,3,978301968,My Fair Lady,"[Musical, Romance]",1964,4.2,F,1,K-12 student,48067
3,1,3408,4,978300275,Erin Brockovich,[Drama],2000,3.9,F,1,K-12 student,48067
4,1,2355,5,978824291,"Bug's Life, A","[Animation, Children's, Comedy]",1998,3.9,F,1,K-12 student,48067
